In [1]:
# slice 작동방식 
class MySeq:
    def __getitem__(self, index):
        return index

s = MySeq()

print(s[1:4])
print(s[1:4:2])
print(s[1:4:2, 9])
print(s[1:4:2, 7:9])

slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [2]:
print(slice)
print(dir(slice))

<class 'slice'>
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']


In [3]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



In [3]:
# 슬라이스를 인식하는 __getitem__()

from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        # 벡터 요소를 배열로 저장
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음

In [8]:
v7 = Vector(range(7))
print(v7[1:4])
print(v7[-1:])
print(v7[1,2]) # 다차원 인덱싱 지원 안함

(1.0, 2.0, 3.0)
(6.0,)


TypeError: Vector indices must be integers

## 동적 속성 접근 
### v.x, v.y 처럼 벡터 요소를 이름으로 접근하는 기능 구형
### __getattr__() 메서드를 이용하여 깔끔하게 구현 

In [9]:
# vector_v3

from array import array
import reprlib
import math
import numbers


class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    # seq protocol
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self) #객체 클래스
        if isinstance(index, slice):
            return cls(self._components[index]) #슬라이스를 받으면, 벡터 객체 생성하여 리턴
        elif isinstance(index, numbers.Integral):
            return self._components[index] # 정수형의 경우, 항목반환
        else : 
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos <len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음


In [15]:
v = Vector(range(5))
print(v)
print(dir(v))
print(v.x,v.y)
print(v.z)
v.x = 10 # v객체에 x속성이 추가되어, __getattr__() 을 사용하지 않음
print(v.x)
print(v) # 벡터 요소는 변경되지 않음

(0.0, 1.0, 2.0, 3.0, 4.0)
['__abs__', '__bool__', '__bytes__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_components', 'frombytes', 'shortcut_names', 'typecode']
0.0 1.0
2.0
10
(0.0, 1.0, 2.0, 3.0, 4.0)


In [17]:
dir(v)[-1]

'x'

In [4]:
# vector_v4

from array import array
import reprlib
import math
import numbers


class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    # seq protocol
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self) #객체 클래스
        if isinstance(index, slice):
            return cls(self._components[index]) #슬라이스를 받으면, 벡터 객체 생성하여 리턴
        elif isinstance(index, numbers.Integral):
            return self._components[index] # 정수형의 경우, 항목반환
        else : 
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos <len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)

        super().__setattr__(name, value) # 에러가 없을 경우 슈퍼클래스 매서드 호출

    def __iter__(self):
        return iter(self._components) # 반복할 수 있도록 구현
    
    def __repr__(self):
        components = reprlib.repr(self._components) # 제한된 길이로 출력
        components = components[components.find('['):-1] # 문자열 중 앞에 나오는 "array('d'," 를 제거
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # 언패킹할 필요가 없음


In [27]:
v = Vector(range(5))
print(v)
print(v.x, v.y)
v.x = 10 # 오류발생

(0.0, 1.0, 2.0, 3.0, 4.0)
0.0 1.0


AttributeError: readonly attribute 'x'

In [31]:
v.test = 10
dir(v)[-4:] # 이러면 적용 가능

['frombytes', 'shortcut_names', 'test', 'typecode']

In [32]:
# 동작 불일치 해결을 위해서는 __getattr__ 과 __setattr__ 을 함께 구현해야함
# 벡터 요소 변경의 두가지 방법
# __setitem__ --> v[0] = 10 
# __setattr__ --> v.x = 10 

## 해시 및 더 빠른 == 연산
### __hash__, __eq__ 매서드를 구현하여 객체가 해시 가능하도록 한다.

In [33]:
# xor 누적계산
n = 0 
for i in range(6):
    n ^= i
print(n)

import functools

v = functools.reduce(lambda a, b: a^b, range(6))
print(v)

import operator
v = functools.reduce(operator.xor, range(6))
print(v)

1
1
1


In [39]:
hash('abc')

-7434516498685352469

In [11]:
import itertools

class Vector_v4(Vector):
    def __eq__(self, other):
        # return tuple(self) == tuple(other)
        if len(self) != len(other): # zip 이 짧은 피연산자 기준으로 멈추기 때문에 길이 우선 검사
            return False
        for a, b in zip(self, other): # 제너레이터로부터 하나씩 비교
            if a != b:
                return False
        return True

    # all 사용하여 구현
    def __eq__(self, other):
        return len(self) == len(other) and all(a==b for a,b in zip(self, other))

    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)

    # 포메팅의 구현 
    def angle(self, n):
        r = math.sqrt(sum(x*x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1]<0):
            return math.pi * 2 - a
        else :
            return a
    
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'

        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

In [12]:
v3 = Vector_v4([3,4,5])
print(format(v3))
print(format(Vector_v4(range(7))))

(3.0, 4.0, 5.0)
(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)


In [14]:
# 구면 좌표 테스트
print( format(Vector_v4([1,1]), 'h'))
print( format(Vector_v4([0,1,0,0]), '0.5fh') )

<1.4142135623730951, 0.7853981633974483>
<1.00000, 1.57080, 0.00000, 0.00000>
